In [1]:
import xarray as xr
import numpy as np
import warnings
import matplotlib.pyplot as plt
import os
import seaborn as sns
from shapely.geometry import box, mapping
import geopandas as gpd
import global_land_mask
import logging
import CMIP6_area_calculations

logging.getLogger('xclim').setLevel('ERROR')
logging.getLogger('root').setLevel('ERROR')

In [2]:
def convert_to_180(ds):
 #   ds=ds.assign_coords(lat=ds.y)
    return (ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))).sortby('lon')

def convert_time(ds):
    if not ds.indexes["time"].dtype in ["datetime64[ns]"]:

        time_objects = ds.indexes['time'].to_datetimeindex() 
        ds=ds.assign_coords({"time": time_objects})                   
        ds = xr.decode_cf(ds)
        
    return ds

def create_land_ocean_mask(cmip6_grid: xr.Dataset) -> xr.DataArray:
    print("[create_land_ocean_mask] Running create_land_ocean_mask")
    lon = cmip6_grid.lon.values
    lat = cmip6_grid.lat.values
    lon_180 = xr.where(lon > 180, lon - 360, lon)
    lon_grid, lat_grid = np.meshgrid(lon_180, lat)

    mask_data = global_land_mask.globe.is_ocean(lat_grid, lon_grid).astype(int)
    #  mask_data[np.isnan(mask_data)] = np.nan

    return xr.DataArray(mask_data, coords={'lat': lat, 'lon': lon},
                        dims=['lat', 'lon'])

def resample_to_monthly(ds:xr.Dataset, var_name):
    ds = xr.where(((ds < 1.e-20) | (ds > 1e20)), np.nan, ds)
    ds = convert_to_180(ds)
    ds = ds.sel(time=slice(start_time,end_time))

    ds_lme = convert_time(ds)
    # Add land mask
    ds_lme["mask"] = create_land_ocean_mask(ds_lme)
    ds_lme = ds_lme.where(ds_lme.mask == 1)
    ds_lme = get_data_within_LME(ds_lme, var_name, LME, False)
    return ds_lme.resample(time="M").mean()

def get_area_averaged_ds(fname_perc, fname_stats,  scenario, var_name, LME):

    var_name_mean = f"{var_name}_mean"

    if os.path.exists(fname_stats) and os.path.exists(fname_perc):
        ds_stats = xr.open_dataset(fname_stats)
        ds_perc = xr.open_dataset(fname_perc)

        ds_stats = resample_to_monthly(ds_stats, var_name_mean)
        ds_perc = resample_to_monthly(ds_perc, var_name)

        # First we calculate the total area covered by the LME. The total areacello is broadcasted along time
        # axis and used later.
        ds_perc["areacello"] = CMIP6_area_calculations.calculate_areacello(ds_perc.sel(percentiles=2.5), var_name)
        total_area_lme = np.nansum(ds_perc["areacello"])

        print("[calculate_areacello] Average total area {:,.2f} km2".format(total_area_lme))

        # We assess where light more than light_threshold (W/m2) is changing. For Polar cod, light_threshold is
        # the minimum light level required to feed. If the area where minimum light increases, this can be indicative
        # of areas opening up for feeding given that food levels are increasing too.
        light_threshold=0.1
        if var_name in ["uvb_mean"]:
            light_threshold=0.001

        ds_perc = get_data_within_LME(ds_perc, var_name, LME, False)
        ds_stats = get_data_within_LME(ds_stats, var_name_mean, LME, False)

        # Average data spatially within the LME as a function of time
        ds = ds_perc.mean({"lat","lon"}, skipna=True)
        ds_s = ds_stats.mean({"lat","lon"}, skipna=True)
        ds_s = ds_s.assign(std = ds_stats[f"{var_name}_stdev"].mean(dim={"lat","lon"}))

        # Convert datasets to Pandas dataframes
        df = ds.to_dataframe().dropna()
        df = df.reset_index()

        df_s = ds_s.to_dataframe().dropna()
        df_s = df_s.reset_index()
        df_s["model_scenario"]=scenario

 # Save to CSV file
        df_s.to_csv(f"Light_data_stats_{LME}_{scenario}.csv")
        df.to_csv(f"Light_data_perc_{LME}_{scenario}.csv")
        ds_stats.close()
        ds_perc.close()
        print(f"Finished with CSV {LME}_{scenario}")
        return

In [3]:
def get_LME_records():
    lme_file='/Users/trondkr/DropBox/NIVA/oceanography/Shapefiles/LME66/LMEs66.shp'
    return gpd.read_file(lme_file)

def get_LME_records_plot():
    lme_file='/Users/trondkr/DropBox/NIVA/oceanography/Shapefiles/LME66_180/LME66_180.shp'
    return gpd.read_file(lme_file)

def get_data_within_LME(ds,var_name,LME,create_maps):
    print("Working on LME: {}".format(LME))

    # Extract the polygon defining the boundaries of the LME
    shdf = get_LME_records()
    #for name in shdf['LME_NAME']:
    #    print(name)
    shdf_sel = shdf[ shdf['LME_NAME']==LME ]

    # Rioxarray requires x and y dimensions - we convert these back to lon and lat later.
    # We also add the projection (lat-lon) so that rioxarray can do the clipping of the data according to the
    # shapefile.
    tos=ds.rename({'lon': 'x','lat': 'y'})
    tos=tos.rio.write_crs(4326)

    # Clip the data within the LME. We have to convert the polygon geometry to a geodataframe using
    # `shapely.geometry`. The clipping of data within the polygon is done using rioxarray.clip function
    clipped = tos.rio.clip(geometries=shdf_sel.geometry.apply(mapping), crs=tos.rio.crs)
    clipped=clipped.rename({'x': 'lon','y': 'lat'}) #.to_dataset()

    return clipped

In [4]:
scenarios = ["ssp245", "ssp585"]
member_range = 12
frequency = "A"
period = "1979-01-01-2099-12-16"
start_time = "1979-01-01"
end_time = "2099-12-16"

models = ["ensemble"]
ds_var_names = ["par"] #, "uvb", "uv"]
#ds_var_names = ["uv","uvb"]  #,"uv"]
write_stats_to_file = False


# These are the regions we extract to fulfill the ACTEA NOAA contract
# 31.08.2022
LMES = ['Gulf of Alaska', 'Northern Bering - Chukchi Seas', 'East Bering Sea', 'Beaufort Sea']

for var_name in ds_var_names:
    for LME in LMES:
        for scenario in scenarios:

            fname_perc = f"/Users/trondkr/DropBox/NIVA/oceanography/light/ncfiles/ensemble/{var_name}_ensemble_perc_" \
                         f"{scenario}.nc"
            fname_stats = f"/Users/trondkr/DropBox/NIVA/oceanography/light/ncfiles/ensemble/{var_name}_ensemble_stats_{scenario}.nc"

            get_area_averaged_ds(fname_perc, fname_stats, scenario, var_name, LME)



[create_land_ocean_mask] Running create_land_ocean_mask
Working on LME: Gulf of Alaska
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian Shelf
F

/Users/trondkr/anaconda3/envs/cmip6/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


[calculate_areacello] Average total area 2,996,447.57 km2
Working on LME: Gulf of Alaska
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian Shelf

/Users/trondkr/anaconda3/envs/cmip6/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


[calculate_areacello] Average total area 2,996,447.57 km2
Working on LME: Gulf of Alaska
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian Shelf

...

[calculate_areacello] Average total area 24,185,131.43 km2
Working on LME: East Bering Sea
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian She

/Users/trondkr/anaconda3/envs/cmip6/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


[calculate_areacello] Average total area 2,254,970.66 km2
Working on LME: Beaufort Sea
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian Shelf
F

/Users/trondkr/anaconda3/envs/cmip6/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


[calculate_areacello] Average total area 2,254,970.66 km2
Working on LME: Beaufort Sea
Baltic Sea
Sea of Okhotsk
North Sea
Gulf of Alaska
Labrador - Newfoundland
Celtic-Biscay Shelf
Sea of Japan
Oyashio Current
Scotian Shelf
California Current
Black Sea
Northeast U.S. Continental Shelf
Mediterranean Sea
Iberian Coastal
Kuroshio Current
Yellow Sea
Canary Current
East China Sea
Southeast U.S. Continental Shelf
Gulf of California
Gulf of Mexico
Arabian Sea
Red Sea
Insular Pacific-Hawaiian
Caribbean Sea
South China Sea
Bay of Bengal
Pacific Central-American Coastal
Sulu-Celebes Sea
Gulf of Thailand
North Brazil Shelf
Guinea Current
Somali Coastal Current
Indonesian Sea
East Brazil Shelf
Humboldt Current
Benguela Current
North Australian Shelf
Northeast Australian Shelf
Agulhas Current
Northwest Australian Shelf
East Central Australian Shelf
South Brazil Shelf
New Zealand Shelf
Patagonian Shelf
Southeast Australian Shelf
Antarctica
South West Australian Shelf
West Central Australian Shelf
F